# Finalize figures



In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scgen
from scipy import stats
import torch
import anndata as ad
import sys
from anndata import AnnData
sys.path.append("/work/users/mh823zote/projects/cov/integration/analysis")
import helper_VAE as hVAE
import random
import matplotlib.pyplot as plt
import scvi
import os
import seaborn as sns
import harmonypy as hpy
import scipy.stats as scistats
from statsmodels.multivariate.manova import MANOVA
from scipy.stats import gaussian_kde
save_location = '/work/users/mh823zote/projects/cov/integration/run_VAE_228_1/backup_results/results_v228_1/ma/Neutrophils_acurate'
celltype = 'Neutrophils_acurate'
experiment_name = 'B4_run_3'
species = 'ma'



# Calculate umap

In [ ]:
model = scgen.SCGEN.load(save_location + '/' + experiment_name + '/model/model.pt')
adata = model.adata

adata_latent = hVAE.get_latent_representation_object(model = model,adata=adata)

adata_latent.obs['compare_umap'].value_counts() 

lat_hamster_control = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='d0')
lat_human_control = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='0')
delta = hVAE.get_delta_in_latent_space(lat_hamster_control,lat_human_control)

d0 = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='d0')
d2 = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='d2')
W0 = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='0')
W5 = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='5')
W7 = hVAE.filter_adata_obs(adata=adata_latent,col_name='compare_umap',val='7')

d2_shifted = hVAE.shift_adata_in_latent_space(d2,delta)
d2_shifted.obs['compare_umap'] = 'd2 shifted'

lat_controls = hVAE.merge_adata(d0,W0)

lat_controls.obs['compare_umap'].value_counts() 

hVAE.prepare_umap(lat_controls)

#sc.pl.umap(lat_controls,color = 'compare_umap')

lat_infected = hVAE.merge_adata(hVAE.merge_adata(hVAE.merge_adata(d2,W5),W7),d2_shifted)

hVAE.prepare_umap(lat_infected)

#sc.pl.umap(lat_infected,color ='compare_umap')
lat_infected.write_h5ad('V105_infected_Neutrophils_with_umap.h5ad')

lat_controls.write_h5ad('V105_control_Neutrophils_with_umap.h5ad')


# Pseudotime

In [ ]:
def make_dpt_overview(adata,column_name,column_value,repetitions = 10):
    sc.pp.neighbors(adata)
    sc.tl.diffmap(adata)
    degrees = sorted(pd.unique(adata.obs[column_name]))
    random_keys = random.sample(list(np.flatnonzero(adata.obs[column_name] == column_value)),repetitions)

    df_dict = {}
    for j in range(repetitions):
        pseudotime_dict = {}
        adata_temp = adata.copy()
        root_loc = random_keys[j]
        adata_temp.uns['iroot'] = root_loc
        sc.tl.dpt(adata_temp,copy=False)
        pseudo_list = []
        for DEG in degrees:
            pt_DEG = hVAE.filter_adata_obs(adata_temp,column_name,DEG).obs['dpt_pseudotime']
            pt_rem_inf = list(filter(lambda val: val !=  np.inf, pt_DEG))
            pseudo_list.append(np.mean(pt_rem_inf))
        pseudotime_dict[column_value] = pseudo_list
        df_dict[j] = pd.DataFrame(pseudotime_dict,index = degrees)
    df_final = df_dict[0].copy()/repetitions
    for q in range(1,repetitions):
        df_final += df_dict[q]/repetitions

    df_dpt_overview = pd.DataFrame(columns = df_dict[0].T.columns)
    for q in range(1,repetitions):
        df_dpt_overview = df_dpt_overview.append(df_dict[q].T)
    return df_dpt_overview,df_final


In [ ]:
lat_dpt = hVAE.leave_out_adata_obs(adata=lat_infected,col_name='compare_umap',val='d2')

#lat_dpt.obs['compare_umap']

df_dpt_overview,df_final = make_dpt_overview(adata = lat_dpt,
                                    column_name = 'compare_umap',
                                    column_value = 'd2 shifted')

df_d2 = df_final[0:2]

dpts = df_d2.values[:,0]
min_val = min(dpts)
max_val = max(dpts)
vis_shift = 0.1*(max_val - min_val)
closeness = (dpts - min_val) + vis_shift
closeness_score = 1-closeness/closeness.sum(0)
df_d2['closeness_score'] = closeness_score

df_d2['day'] = 'd2'
df_d2['cov_degree'] = df_d2.index

df_d2_pivot = df_d2.pivot("day", "cov_degree", "closeness_score").astype(float)



In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
sns.set(style='ticks')

#subfigure 1
df_melted = pd.melt(df_dpt_overview, value_vars=['5', '7'])
# Create a boxplot using Seaborn
sns.boxplot(x='variable', y='value', data=df_melted,width=0.5,ax = axes[0],palette=['#FF7878', '#8B0000'])
sns.stripplot(data=df_melted,x='variable', y='value', color='black',ax = axes[0], size=2, jitter=False,order = ['5','7'])
axes[0].set_ylim([0.1,0.35])
axes[0].set_xlabel('WHO severity', fontsize=12)
axes[0].set_ylabel('Diffusion pseudotime (dpt)', fontsize=12)
axes[0].set_title('Mean dpt distance per root cell from day 2')
axes[0].grid(True,alpha = 0.3)
for i in range(len(df_dpt_overview)):
    axes[0].plot([0, 1], [df_dpt_overview['5'][i], df_dpt_overview['7'][i]], '--o', color='grey', linewidth=1)



#subfigure 2
sns.heatmap(df_d2_pivot,cmap = "Reds",annot=True, ax = axes[1])

#axes[1].yticks(rotation = 0)
axes[1].set_xlabel('WHO degree',fontsize = 12)
axes[1].set_ylabel('timepoint',fontsize = 12)
axes[1].set_title('Similarity score from dpt')
plt.savefig('V105_dpt_heatmaps_Neutrophils.pdf')

